In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

theta = 'pi/4'

folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

Purify = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

In [ ]:
all_lt3,all_lt4,offsets,offsets_ch1 = Purify.get_tstamps_and_offsets(contains = 'Purify', verbose = False, return_tstamps = True)

In [ ]:
Purify.calculate_sequence_time(all_lt4,st_start= 2772.5e3,st_len=40e3,max_w2 = 150)